Import statements

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

Checking the no of samples and shape of samples.

In [6]:
digits = load_digits()
print(digits.data.shape)
X,y = digits.data, digits.target
n = X.shape[0]
print(n)
print(digits.images[0])

(1797, 64)
1797
[[ 0.  0.  5. 13.  9.  1.  0.  0.]
 [ 0.  0. 13. 15. 10. 15.  5.  0.]
 [ 0.  3. 15.  2.  0. 11.  8.  0.]
 [ 0.  4. 12.  0.  0.  8.  8.  0.]
 [ 0.  5.  8.  0.  0.  9.  8.  0.]
 [ 0.  4. 11.  0.  1. 12.  7.  0.]
 [ 0.  2. 14.  5. 10. 12.  0.  0.]
 [ 0.  0.  6. 13. 10.  0.  0.  0.]]


As the pixel values range from 0-15 we should normalize for easier and faster computation

In [7]:
data =np.array(digits.data)
data = data/15
print(data)

[[0.         0.         0.33333333 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.66666667 0.         0.        ]
 [0.         0.         0.         ... 1.06666667 0.6        0.        ]
 ...
 [0.         0.         0.06666667 ... 0.4        0.         0.        ]
 [0.         0.         0.13333333 ... 0.8        0.         0.        ]
 [0.         0.         0.66666667 ... 0.8        0.06666667 0.        ]]


Split data into training and testing (70,30)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape,X_test.shape)

(1257, 64) (540, 64)


Function to obtain the class conditional probabilities for the Naive Bayes Classifier to perform training and predicition. Sigma will the hyperparameter which we can run a optimiziation for.

In [12]:
def kde_scratch(sigma, D_A, D_B):
    getcontext().prec = 7
    mu, prob_x = D_A.astype(np.float64), 0
    len_D_A, len_D_B, d = len(D_A), len(D_B), len(D_A[0])
    t_1 = -Decimal(0.5 * d) * Decimal(2 * pi * (sigma ** 2)).ln()
    log_k = Decimal(len_D_A).ln()

    for i in range(0, len_D_A):
        t_0 = np.sum((-((np.matlib.repmat(D_B[i], len_D_A, 1).astype(np.float64) - mu) ** 2)) / (2 * (sigma ** 2)),
                     axis=1)
        elements_sum = 0
        for j in range(0, len_D_B):
            elements_sum += Decimal(t_0[j]).exp()
        prob_x += t_1 - log_k + elements_sum.ln()
    return prob_x / len_D_B

Class for writing the NaiveBayes

In [ ]:
class NaiveBayesClassifier():
    def calc_prior(self, features, target):
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()

        return self.prior
    
    def calc_statistics(self, features, target):
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
        return self.mean, self.var
    
    def gaussian_density(self, class_idx, x):     
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob
    
    def RbfKernel(data1, data2, sigma):
        delta =abs(np.subtract(data1, data2))
        squaredEuclidean = (np.square(delta).sum(axis=1))
        result = np.exp(-(squaredEuclidean)/(2*sigma**2))
        return result

    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.Kde_scratch(f) for f in features.to_numpy()]
        return preds